# Assignment 3 - Blood Managment

Note: you will need packages `cvxopt` and `glpk`for this notebook.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import time
import cvxopt
from collections import (namedtuple, defaultdict)
import os.path
import os
import warnings
warnings.filterwarnings("ignore")
# %matplotlib ipympl

In [2]:
def initLPMatrices(params,Bld_Net):
    #Initializing the matrix for the LP
    A = np.zeros((params['NUM_BLD_NODES'], params['NUM_BLD_NODES']*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])))
    for i in range(params['NUM_BLD_NODES']):
        for j in range(params['NUM_BLD_NODES']*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])):
            if (j < (i+1)*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])) and (j >= i*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])):
                #Checking for feasibility
                k=j-i*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])
                if (k<params['NUM_DEM_NODES']):
                    bloodnode = Bld_Net.bloodnodes[i]
                    demandnode = Bld_Net.demandnodes[k]
                    if (demandnode[2] == False and bloodnode[0] == demandnode[0]) or (demandnode[2] == True and params['SubMatrix'][(bloodnode[0], demandnode[0])] == True):
                        A[i,j] = 1
                else:   
                    A[i,j] = 1    


    G = np.zeros((params['NUM_DEM_NODES'] + 2*params['NUM_BLD_NODES']*params['NUM_PARALLEL_LINKS'] + params['NUM_DEM_NODES']*params['NUM_BLD_NODES'], params['NUM_BLD_NODES']*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])))
    # ineq constr for sum x_tbd < D_td
    for i in range(params['NUM_DEM_NODES']):
        for j in range(params['NUM_BLD_NODES']*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])):
            if (j % (params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS']) == i):
                G[i,j] = 1.
    # ineq constr for parallel links <= SLOPE_CAPAC
    for i in range(params['NUM_BLD_NODES']):
        for j in range(params['NUM_PARALLEL_LINKS']):
            G[params['NUM_DEM_NODES'] + i*params['NUM_PARALLEL_LINKS'] + j, (params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])*i + params['NUM_DEM_NODES'] + j] = 1

    # ineq constr for x_tbd >= 0
    for i in range(params['NUM_BLD_NODES']):
        for j in range(params['NUM_DEM_NODES']):
            G[params['NUM_DEM_NODES'] + params['NUM_BLD_NODES']*params['NUM_PARALLEL_LINKS'] + i*params['NUM_DEM_NODES'] + j, (params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])*i + j] = -1


    # ineq constr for x_parallel >= 0
    for i in range(params['NUM_BLD_NODES']):
        for j in range(params['NUM_PARALLEL_LINKS']):
            G[params['NUM_DEM_NODES'] + params['NUM_BLD_NODES']*params['NUM_PARALLEL_LINKS'] + params['NUM_DEM_NODES']*params['NUM_BLD_NODES'] + i*params['NUM_PARALLEL_LINKS'] + j,(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])*i + params['NUM_DEM_NODES'] + j] = -1


    h = np.ones(params['NUM_DEM_NODES'] + params['NUM_BLD_NODES']*params['NUM_PARALLEL_LINKS'])
    h[params['NUM_DEM_NODES']::params['NUM_PARALLEL_LINKS']]= params['SLOPE_CAPAC_LAST']
    h = np.append(h, np.zeros(params['NUM_BLD_NODES']*params['NUM_DEM_NODES'] + params['NUM_BLD_NODES']*params['NUM_PARALLEL_LINKS']))
        
    A = cvxopt.matrix(A)
    G = cvxopt.matrix(G)

    coeff = [np.concatenate((np.array(Bld_Net.demcontrib[bld]),np.zeros(params['NUM_PARALLEL_LINKS']))) if int(bld[1])< params['MAX_AGE']-1 else np.concatenate((np.array(Bld_Net.demcontrib[bld]),np.add(np.zeros(params['NUM_PARALLEL_LINKS']),params['DISCARD_BLOOD_PENALTY'])))  for bld in Bld_Net.bloodnodes]
    coeff = [ai for a in coeff for ai in a ]    
    coeff = np.array(coeff)

    return (A,G,h,coeff)

class Policy():
    """
    Base class for Static Stochastic Shortest Path Model policy
    """

    def __init__(self,params,Bld_Net):
        """
        Initializes the policy
        """

        self.A,self.G,self.h,self.coeff = initLPMatrices(params,Bld_Net)
    

    def getLPSol(self,params,M,iteration,t,solDemList,solHoldList,IS_TRAINING):
       
        c_t = [np.concatenate((np.multiply(np.array(M.Bld_Net.demcontrib[bld]),-1),np.multiply(M.Bld_Net.parallelarr[(t, bld, M.Bld_Net.supersink)],-params['DISCOUNT_FACTOR']))) if int(bld[1])< params['MAX_AGE']-1 else np.concatenate((np.multiply(np.array(M.Bld_Net.demcontrib[bld]),-1),np.add(np.multiply(M.Bld_Net.parallelarr[(t, bld, M.Bld_Net.supersink)],-params['DISCOUNT_FACTOR']),-params['DISCARD_BLOOD_PENALTY'])))  for bld in M.Bld_Net.bloodnodes]
        c = [ai for a in c_t for ai in a ]
        b = np.array(M.Bld_Net.bloodamount)
        self.h[:params['NUM_DEM_NODES']] = M.Bld_Net.demandamount

        
        c = cvxopt.matrix(c)
        b = cvxopt.matrix(b,size=(params['NUM_BLD_NODES'],1),tc='d')
        h = cvxopt.matrix(self.h)
        
        cvxopt.solvers.options['show_progress'] = False
        sol = cvxopt.solvers.lp(c, self.G, h, self.A, b,solver='glpk',options={'glpk':{'msg_lev':'GLP_MSG_OFF'}}) 
        #sol = cvxopt.solvers.lp(c, self.G, h, self.A, b) 

        x = sol['x']
           
        x = np.array(x)
        x = np.squeeze(x)
        
        val = np.dot(x, self.coeff)
        
        
        
        xDem = [x[i*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS']):i*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])+params['NUM_DEM_NODES']] for i in list(range(params['NUM_BLD_NODES']))]
        xDemFlat = [xij for xi in xDem for xij in xi]
        solDemRec=(iteration,t,xDem.copy())
        solDemList.append(solDemRec)
        
        hld=[np.sum(x[i*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])+params['NUM_DEM_NODES']:(i+1)*(params['NUM_DEM_NODES']+params['NUM_PARALLEL_LINKS'])]) for i in list(range(params['NUM_BLD_NODES']))]
        solHoldRecord = (iteration,t,hld.copy())
        solHoldList.append(solHoldRecord)
        hld = np.array(hld)
        
        invByBlood = [np.sum(M.bld_inv[i*params['MAX_AGE']:(i+1)*params['MAX_AGE']]) for i in list(range(len(params['Bloodtypes']))) ]
        demByBlood = [np.sum(M.Bld_Net.demandamount[i*(len(params['Surgerytypes'])*len(params['Substitution'])):(i+1)*(len(params['Surgerytypes'])*len(params['Substitution']))]) for i in list(range(len(params['Bloodtypes']))) ]
        
        xDemFlat = [xij for xi in xDem for xij in xi]
        xDemMat = np.array(xDemFlat).reshape(params['NUM_BLD_NODES'],params['NUM_DEM_NODES'])
        xDemMatColSum =  xDemMat.sum(axis=0)   
        covByBlood = [ np.sum(xDemMatColSum[i*(len(params['Surgerytypes'])*len(params['Substitution'])):(i+1)*(len(params['Surgerytypes'])*len(params['Substitution']))]) for i in list(range(len(params['Bloodtypes']))) ]
        covByBlood = np.array(covByBlood).astype(int)
        
        hldByBlood = [int(np.sum(hld[i*params['MAX_AGE']:(i+1)*params['MAX_AGE']])) for i in list(range(len(params['Bloodtypes']))) ]
        disByBlood = hld[params['MAX_AGE']-1::params['MAX_AGE']]
        disByBlood = np.array(disByBlood)
        disByBlood = disByBlood.astype(int)
        
        if False:
            print('Iteration = ', iteration)
            res_dict('Time period = ', t)
            res_dict('Demand = ', np.sum(M.Bld_Net.demandamount))
            res_dict('Supply = ', np.sum(M.Bld_Net.bloodamount))
            res_dict('Blood Used = ', np.sum(M.bld_inv) - np.sum(hld))
            print('Blood Held = ', np.sum(hld))
            print('Inventory by BloodType ',invByBlood)
            print('Demand By BloodType ',demByBlood)
            print('Used By BloodType ', list(covByBlood))
            print('Hold By BloodType ',hldByBlood)
            print('Discard By BloodType ', list(disByBlood))
            print('Contribution = ', val)   
            print('Donation = ', np.sum(M.donation)) 
            print('\n')
        
        hld = hld.astype(int)
        
        if IS_TRAINING and params['IS_PERTUB']:
            epsilon = PERTUB_GEN.poisson(LAMBDA_PERTUB, params['NUM_BLD_NODES']) 
            signE = PERTUB_GEN.choice([-1,1], size=params['NUM_BLD_NODES'], replace=True, p=None)
            hld = hld+epsilon*signE
            hld = np.maximum(np.zeros(params['NUM_BLD_NODES']),hld)
            hld = hld.astype(int)
    
        # dual variables
        d = sol['y']
               
        return sol,val,x,hld,d,solDemList,solHoldList
    
    def updateVFAs(self,params,M,iteration,t,d, slopesList,updateVfaList):
        alpha = 0

        # set the dual variables to respective parallel arcs
        for i in range(params['NUM_BLD_NODES']):
            # put the value of the dual varible d[i+1] in the parallel arc, associated 
            # with the amount of resource in the inventory associated with holdnode[i]
            # the holdnodes with the oldest age do not get updated 
            
            recordSlopes = (iteration,t,M.Bld_Net.parallelarr[(t, M.Bld_Net.holdnodes[i], M.Bld_Net.supersink)].copy())
            slopesList.append(recordSlopes)
        
            index = M.bld_inv[i]
            if index>=0:
                if (t>0 and M.Bld_Net.holdnodes[i][1]<str(params['MAX_AGE']-1)):
                    vhat=d[i+1]
                    
                    if index >= params['NUM_PARALLEL_LINKS'] - 1:
                        index = params['NUM_PARALLEL_LINKS'] - 1
                        
                    arr = M.Bld_Net.varr[(t-1,M.Bld_Net.holdnodes[i], M.Bld_Net.supersink)]
                    sqGradArr = M.Bld_Net.sqGrad[(t-1,M.Bld_Net.holdnodes[i])]
                    

                    if iteration < params['NUM_ITER_STEP_ONE']:
                            alpha = 1
                    else:
                        if (params['STEPSIZE_RULE'] == 'C'):
                            alpha = params['ALPHA']
                        elif (params['STEPSIZE_RULE'] == 'A'):
                            sqGradArr[index] += np.power(vhat-arr[index],2) 
                            alpha = params['ETA']/(np.sqrt(sqGradArr[index]+params['STEP_EPS']))
                   
                    vbar = arr[index]
                    vnew = alpha*vhat +(1-alpha)*vbar
                    arr[index] = vnew
                    
                    recordUpdateVfa = (iteration,t-1,M.Bld_Net.holdnodes[i][0],M.Bld_Net.holdnodes[i][1],index,vhat,vbar,sqGradArr[index],alpha,vnew)
                    updateVfaList.append(recordUpdateVfa)

                    #Projecting back in case the vfa is not concave anymore
                    if (vnew>vbar): #Look to the left
                        indSetL=[i for i in list(range(0,index+1)) if arr[i]<=vnew]
                        if (len(indSetL)>0):
                            if params['PROJECTION_ALGO'] == 'Avg':
                                avg = np.mean(arr[indSetL])
                                arr[indSetL]=avg
                            elif params['PROJECTION_ALGO'] == 'Copy':
                                arr[indSetL]=vnew 
                            else:
                                if index > 0:
                                    j=index-1
                                    while (j>=0 and arr[j] < arr[j+1]):
                                        arr[j]= alpha*vhat +(1-alpha)*arr[j]
                                        j-=1
                                else:
                                    arr[index]=vnew 


                            
                    elif (vnew<vbar): #Look to the right
                        indSetR=[i for i in list(range(index,params['NUM_PARALLEL_LINKS'])) if arr[i]>=vnew]
                        if (len(indSetR)>0):
                            if params['PROJECTION_ALGO'] == 'Avg':
                                avg = np.mean(arr[indSetR])
                                arr[indSetR]=avg
                            elif params['PROJECTION_ALGO'] == 'Copy':   
                                arr[indSetR]=vnew
                            else:
                                if index < params['NUM_PARALLEL_LINKS']-1:
                                    j=index+1
                                    while (j<params['NUM_PARALLEL_LINKS'] and arr[j] > arr[j-1]):
                                        arr[j] = alpha*vhat +(1-alpha)*arr[j]
                                        j+=1
                                else:
                                    arr[index]=vnew 

        return alpha,slopesList,updateVfaList

class Model():
    """
    Base class for model
    """

    def __init__(self, state_names, x_names, init_state, Bld_Net,params):
        
        self.params = params
        self.init_state = init_state
        self.state_names = state_names
        self.x_names = x_names
        self.State = namedtuple('State', state_names)
        self.state = self.build_state(init_state)
        self.Decision = namedtuple('Decision', x_names)
        self.obj = 0.0
        self.Bld_Net = Bld_Net
        self.bld_inv = init_state['BloodInventory']
        self.demand = init_state['Demand']
        self.donation = init_state['Donation']
        
        # include initial inventory into the network
        for i in range(self.params['NUM_BLD_NODES']):
            self.Bld_Net.bloodamount[i] = self.bld_inv[i]
        
        # include initial demand into the network
        for i in range(self.params['NUM_DEM_NODES']):
            self.Bld_Net.demandamount[i] = self.demand[i]

        
    def build_state(self, info):
        return self.State(*[info[k] for k in self.state_names])

    def build_decision(self, info):
        return self.Decision(*[info[k] for k in self.x_names])
    
    # exogenous information = demand from t-1 to t and new donated blood
    def exog_info_fn(self, exog_info):
        self.demand  = exog_info.demand
        # update the demand nodes
        for i in range(self.params['NUM_DEM_NODES']):
            self.Bld_Net.demandamount[i] = exog_info.demand[i]
        # save the donation vector to the model
        self.donation = exog_info.donation
        return exog_info
    
    def transition_fn(self, decision):
        # iterate through hold vector
        hold = decision[0]
        for i in range(self.params['NUM_BLD_NODES']):
            self.Bld_Net.holdamount[i] = hold[i]
            
        
        rev_don = list(reversed(self.donation))
        rev_hld = list(reversed(self.Bld_Net.holdamount))
        # age the blood at hold node and add in the donations
        
        for i in range(self.params['NUM_BLD_NODES']):
            if (i % self.params['MAX_AGE'] == self.params['MAX_AGE']-1):
                # add donation
                rev_hld[i] = rev_don[i // self.params['MAX_AGE']]
            else:
                # age
                rev_hld[i] = rev_hld[i+1]
        
        rev_hld = list(reversed(rev_hld))
        # amount at blood node = amount at hold node
        for i in range(self.params['NUM_BLD_NODES']):
            self.Bld_Net.bloodamount[i] = rev_hld[i]  
        
        # updating obj value       
        self.obj += decision[1] 
        
        # update current state
        self.bld_inv = self.Bld_Net.bloodamount
        return self.state

    def objective_fn(self):
        return self.obj
    
########################################################################################################
    
class Exog_Info():
    def __init__(self, demand, donation):
        # list consisting of blood demand objects
        self.demand = demand
        # list consisting of blood unit objects donated to the blood inventory
        self.donation = donation


# function to generate random exogenous information dependent on blood type and time t   
def generate_exog_info_by_bloodtype(t, Bld_Net, params):
    # demand
    demand= []
    if (t in params['TIME_PERIODS_SURGE'] and np.random.uniform(0, 1) < params['SURGE_PROB']):
        factor = params['SURGE_FACTOR']
    else:
        factor = 0
    demand = [round(np.random.uniform(0, params['MAX_DEM_BY_BLOOD'][dmd[0]]*params['SURGERYTYPES_PROP'][dmd[1]]*params['SUBSTITUTION_PROP'][dmd[2]])) + factor*int(np.random.poisson(params['MAX_DEM_BY_BLOOD'][dmd[0]]*params['SURGERYTYPES_PROP'][dmd[1]]*params['SUBSTITUTION_PROP'][dmd[2]]))  for dmd in Bld_Net.demandnodes]    
    
    # donation
    donation = [round(np.random.uniform(0, params['MAX_DON_BY_BLOOD'][i])) for i in params['Bloodtypes']]    
    return Exog_Info(demand, donation)


# function to generate random exogenous information dependent on blood type and time t 
def generate_exog_info_by_bloodtype_p(t, Bld_Net, params):
    # demand
    if (t in params['TIME_PERIODS_SURGE'] and np.random.uniform(0, 1) < params['SURGE_PROB']):
        factor = params['SURGE_FACTOR']
    else:
        factor = 1
    
    demand = [int(np.random.poisson(factor*params['MAX_DEM_BY_BLOOD'][dmd[0]]*params['SURGERYTYPES_PROP'][dmd[1]]*params['SUBSTITUTION_PROP'][dmd[2]])) for dmd in Bld_Net.demandnodes]   

    if False:
        demand=[]
        for dmd in Bld_Net.demandnodes: 
            if dmd[0]=="O-":
                if dmd[1]=="Urgent":
                    demand.append(1)
                else:
                    eleDem=max(0,int(np.random.poisson(factor*params['MAX_DEM_BY_BLOOD'][dmd[0]]-1))-1)
                    demand.append(eleDem)

            else:
                demand.append(int(np.random.poisson(factor*params['MAX_DEM_BY_BLOOD'][dmd[0]]*params['SURGERYTYPES_PROP'][dmd[1]]*params['SUBSTITUTION_PROP'][dmd[2]])))  


    
    #donation
    donation = [int(np.random.poisson(params['MAX_DON_BY_BLOOD'][i])) for i in params['Bloodtypes']]    
    
    return Exog_Info(demand, donation)



##########################################################################################################
# function to calculate one step contribution 
def contribution(params,bloodnode, demandnode):
    
    # if substutition is not allowed
    if (demandnode[2] == False and bloodnode[0] != demandnode[0]) or (demandnode[2] == True and params['SubMatrix'][(bloodnode[0], demandnode[0])] == False):
        value=params['INFEASIABLE_SUBSTITUTION_PENALTY']
    else:
        # start giving a bonus depending on the age of the blood
        #value = params['AGE_BONUS'][int(bloodnode[1])]
        value=0
        # no substitution
        if bloodnode[0] == demandnode[0]:
            value += params['NO_SUBSTITUTION_BONUS']
        # filling urgent demand
        if demandnode[1] == 'Urgent':
            value += params['URGENT_DEMAND_BONUS']
        # filling elective demand
        else:
            value += params['ELECTIVE_DEMAND_BONUS']

        if demandnode[1] == 'Elective':
            value += params['BLOOD_FOR_ELECTIVE_PENALTY']
   
    

    return(value)   


class Graph:
    def __init__(self):
        self.bloodnodes = list()
        self.bloodamount = []
        
        self.demandnodes = list()
        self.demandamount = []
        self.demcontrib = {}
        
        self.demedges = defaultdict(list)
        self.demweights = {}
        
        
        
        self.supersink = None
        
        self.holdnodes = list()
        self.holdamount = []
        self.holdedges = defaultdict(list)
        self.holdweights = {}
        self.holdvbar = []
        
        self.parallelarr = {}
        self.varr = {}
        
        self.sqGrad = {} #this will store the sum of the squared gradients when using AdaGrad stepsizes.
        
    
    # supersink_node
    def add_supersinknode(self, name):
        self.supersink = name
        self.amount[name] = 0
    
    # node of type (bloodtype, age) for current blood inventory
    def add_bloodnode(self, name):
        self.bloodnodes.append(name)
        self.bloodamount.append(0)
    
    # node - (bloodtype, age)
    def add_demandnode(self, name):
        self.demandnodes.append(name)
        self.demandamount.append(0)
    
    # node - (bloodtype, age)
    def add_holdnode(self, name):
        self.holdnodes.append(name)
        self.holdamount.append(0)
    
    # create an edge between two nodes
    def add_demedge(self, from_node, to_node, weight):
        self.demedges[from_node].append(to_node)
        self.demweights[(from_node, to_node)] = weight
        
    # create an edge between two nodes
    def add_holdedge(self, from_node, to_node, weight):
        self.holdedges[from_node].append(to_node)
        self.holdweights[(from_node, to_node)] = weight 
    
    def add_parallel(self, t, from_node, to_node, parallelarray):
        self.parallelarr[(t, from_node, to_node)] =  parallelarray
            
    def add_varr(self, t, from_node, to_node, varr):
        self.varr[(t, from_node, to_node)] = varr
      
    def add_demcontribArr(self, bldnode,demcontribArr):
        self.demcontrib[bldnode] = demcontribArr
    
    def add_sqGradArr(self, t, bldnode,sqGradArr):
        self.sqGrad[(t,bldnode)] = sqGradArr


def create_bld_net(params):
    # create the network
    Bl_Net = Graph()
    Bl_Net.supersink = ('supersink', np.inf)
    # (BloodUnit, Age) pairs and respective hold nodes
    for i in params['Bloodtypes']:
        for j in params['Ages']:
            Bl_Net.add_bloodnode((i, str(j)))
            Bl_Net.add_holdnode((i, str(j)))

    # all possible demand nodes
    for i in params['Bloodtypes']:
        for j in params['Surgerytypes']:
            for k in params['Substitution']:
                Bl_Net.add_demandnode((i, j, k))

    #add edges from (bloodunit, age) pairs to suitable demand nodes
    for bld in Bl_Net.bloodnodes:
        for dmd in Bl_Net.demandnodes:
            weight = contribution(params,bld, dmd)
            Bl_Net.add_demedge(bld, dmd, weight)
    
    for bld in Bl_Net.bloodnodes:
        demcontribArr = [contribution(params,bld, dmd) for dmd in Bl_Net.demandnodes]
        Bl_Net.add_demcontribArr(bld,demcontribArr)


    # add edges from blood nodes to hold nodes
    for bld in Bl_Net.bloodnodes:
        for hld in Bl_Net.holdnodes:
            if bld[0] == hld[0] and bld[1] == hld[1]:
                Bl_Net.add_holdedge(bld, hld, 0)

    # add parallel edges from hold nodes to supersink
    for t in params['Times']:
        for hld in Bl_Net.holdnodes:
            parArr = np.zeros(params['NUM_PARALLEL_LINKS'])
            vArr = np.zeros(params['NUM_PARALLEL_LINKS'])
            Bl_Net.add_parallel(t, hld, Bl_Net.supersink, parArr)
            Bl_Net.add_varr(t, hld, Bl_Net.supersink, vArr)
            
            sqGradArr = np.zeros(params['NUM_PARALLEL_LINKS'])
            Bl_Net.add_sqGradArr(t, hld, sqGradArr)
            
            
            
            
    return(Bl_Net)


In [3]:
def elapsed_since(start):
    return time.strftime("%H:%M:%S", time.gmtime(time.time() - start))


def get_process_memory():
    process = psutil.Process(os.getpid())
    return process.get_memory_info().rss


def track(func):
    def wrapper(*args, **kwargs):
        mem_before = get_process_memory()
        start = time.time()
        result = func(*args, **kwargs)
        elapsed_time = elapsed_since(start)
        mem_after = get_process_memory()
        print("{}: memory before: {:,}, after: {:,}, consumed: {:,}; exec time: {}".format(
            func.__name__,
            mem_before, mem_after, mem_after - mem_before,
            elapsed_time))
        return result
    return wrapper


def printParams(params):
    print(params) 
        

def loadParams(param_dict):

    # parDf = pd.read_excel(filename, sheet_name = 'Parameters')

    # # print(parDf)
    
    # parDict=parDf.set_index('Index').T.to_dict('list')

    parDict = {i: [param_dict[i]] for i in param_dict.keys()}
    
    params = {key:v for key, value in parDict.items() for v in value}

    params['PRINT']=False
    params['PRINT_ALL']=False
    params['OUTPUT_FILENAME'] = 'DetailedOutput.xlsx'

    # params['SHOW_PLOTS']=False




    #Set here bloodtypes and substitutions that are allowed
    params['Bloodtypes'] = ['AB+', 'AB-', 'A+', 'A-','B+', 'B-', 'O+', 'O-'] 
    params['NUM_BLD_TYPES'] = len(params['Bloodtypes'])

    b = [(x,y) for x in params['Bloodtypes'] for y in params['Bloodtypes']]
    f = [False]*(len(params['Bloodtypes'])*len(params['Bloodtypes']))
    c = {k:v for k,v in zip(b, f)}
    #In case we want to allow subs
    c[('AB+', 'AB+')] = True

    c[('AB-', 'AB+')] = True
    c[('AB-', 'AB-')] = True

    c[('A+', 'AB+')] = True
    c[('A+', 'A+')] = True

    c[('A-', 'AB+')] = True
    c[('A-', 'AB-')] = True
    c[('A-', 'A+')] = True
    c[('A-', 'A-')] = True

    c[('B+', 'AB+')] = True
    c[('B+', 'B+')] = True

    c[('B-', 'AB+')] = True
    c[('B-', 'AB-')] = True
    c[('B-', 'B+')] = True
    c[('B-', 'B-')] = True

    c[('O+', 'AB+')] = True
    c[('O+', 'A+')] = True
    c[('O+', 'B+')] = True
    c[('O+', 'O+')] = True

    c[('O-', 'AB+')] = True
    c[('O-', 'A+')] = True
    c[('O-', 'B+')] = True
    c[('O-', 'O+')] = True
    c[('O-', 'AB-')] = True
    c[('O-', 'A-')] = True
    c[('O-', 'B-')] = True
    c[('O-', 'O-')] = True
    params['SubMatrix'] = c

    # Set here max age of blood
    params['MAX_AGE'] = 3
    params['Ages'] = list(range(params['MAX_AGE']))

    params['NUM_BLD_NODES'] = params['NUM_BLD_TYPES'] * params['MAX_AGE']

    # Set here blood demand nodes
    params['Surgerytypes'] = ['Urgent', 'Elective']
    params['Substitution'] = [True]

    params['NUM_SUR_TYPES'] = len(params['Surgerytypes'])
    params['NUM_DEM_NODES'] = params['NUM_BLD_TYPES'] * params['NUM_SUR_TYPES'] * len(params['Substitution'])


    # Solver params
    params['SLOPE_CAPAC_LAST'] = 100000
    params['MIN_CONST'] = 0.01
    params['EPSILON'] = 0.001


    # Set here number of iterations and time periods
    params['NUM_TRAINNING_ITER'] = int(params['NUM_TRAINNING_ITER'])
    params['NUM_TESTING_ITER']=int(params['NUM_TESTING_ITER'])
    params['NUM_ITER'] = int(params['NUM_TESTING_ITER'] + params['NUM_TRAINNING_ITER']) #Total number of iterations
    params['MAX_TIME']=int(15)
    params['Times'] = list(range(params['MAX_TIME']))

    # Set here VFA parameters
    # - If USE_VFA is set to True we are going to use VFA's when making the decisions - 
    # - If USE_VFA is set to False, it means that a  MYOPIC policy is going to be considered and all the parameters
    #related to VFA's (such as DISCOUNT_FACTOR, LOAD_VFA, SAVE_VFA, STEPSIZE_RULE, PROJECTION_ALGO, 
    #IS_PERTUB,SEED_TRAINING are ignored)
    #params['USE_VFA'] = True #If set to True we are going to use VFA's when making the decisions - False means a MYOPIC policy
    params['DISCOUNT_FACTOR'] = 0.95

    params['LOAD_VFA'] = False #If set to True we are going to initialize the VFA's with VFA's from previous runs - instead of all zeros
    params['NAME_LOAD_VFA_PICKLE'] = "Bld_Net10_P_C_Subs.pickle"
    params['SAVE_VFA'] = False #If we want to save/update the VFA's to be used in future runs
    params['NAME_SAVE_VFA_PICKLE'] = "Bld_Net10_P_C_Subs.pickle"

    # Set here the stepsize parameters
    params['STEPSIZE_RULE'] = 'C' #Possible values: 'C' for Constant or 'A' for AdaGrad
    params['NUM_ITER_STEP_ONE'] = 0 #Number of iterations with stepsize one 

    # Set here the CONSTANT stepsize parameter (not considered if AdaGrad stepsize is being used)
    #params['ALPHA'] = 0.2 #the stepsize for the other iterations

    #Set here the AdaGrad stepsize parameters (not considered if Constant stepsize is being used)
    params['STEP_EPS'] = 0.00000001
    params['ETA'] = 1

    # Set here the algorithm that should be use for projection back the slopes that break concavity
    # Possible algorithms for projecting back the slopes to enforce concavity are:
    # - 'Avg' to average the slopes that break concavity; \
    # - 'Copy' to copy the newly updated vbar to the slopes that break concavity
    # - 'Up' to update the slopes that break concavity with the current stepsize and vhat 
    params['PROJECTION_ALGO'] = 'Up' 


    #Perturb the solution during training iterations for exploration
    params['IS_PERTUB'] = False
    params['LAMBDA_PERTUB'] = 1
    params['PERTUB_GEN'] = np.random.RandomState(13247)

    # Set here one step contribution function parameters  - BONUSES and PENALTIES 
    params['AGE_BONUS']=np.zeros(params['MAX_AGE'])
    # params['AGE_BONUS']=[2]*MAX_AGE
    # params['AGE_BONUS']=list(reversed(list(range(0,MAX_AGE))))
    # params['AGE_BONUS']=list(range(0,MAX_AGE))
    # params['AGE_BONUS']=[0.5, 2] #It has to be the same length as MAX_AGE

    params['INFEASIABLE_SUBSTITUTION_PENALTY'] = -500000
    params['NO_SUBSTITUTION_BONUS'] = 0

    # Set here Random Seeds 
    params['SEED_TRAINING'] = 1090377
    params['SEED_TESTING'] = 8090373

    #Set here the distribution for demand/donation/initial inventory
    params['SAMPLING_DIST'] = 'P' #Possible values: 'P' for Poisson or 'U' for uniform
    params['POISSON_FACTOR'] = 1

    # Set here max demand by blood type (when 'U'niform dist) or mean demand (when 'P'oisson dist) 
    params['DEFAULT_VALUE_DIST'] = 20
    d = [params['DEFAULT_VALUE_DIST']] * params['NUM_BLD_TYPES']
    params['MAX_DEM_BY_BLOOD'] = {k:v for k,v in zip(params['Bloodtypes'], d)}
    params['MAX_DON_BY_BLOOD'] = {k:v for k,v in zip(params['Bloodtypes'], d)}


    # Set here demand by blood type (for blood types that are different than the params['DEFAULT_VALUE_DIST'])
    params['MAX_DEM_BY_BLOOD']['AB+'] = 3
    params['MAX_DEM_BY_BLOOD']['B+'] = 9
    params['MAX_DEM_BY_BLOOD']['O+'] = 18
    params['MAX_DEM_BY_BLOOD']['B-'] = 2
    params['MAX_DEM_BY_BLOOD']['AB-'] = 3
    params['MAX_DEM_BY_BLOOD']['A-'] = 6
    params['MAX_DEM_BY_BLOOD']['O-'] = 7
    params['MAX_DEM_BY_BLOOD']['A+'] = 14

    #params['DEFAULT_VALUE_DIST']

    # Set here donation by blood type (for blood types that are different than the params['DEFAULT_VALUE_DIST'])
    params['MAX_DON_BY_BLOOD']['AB+'] = 3
    params['MAX_DON_BY_BLOOD']['B+'] = 9
    params['MAX_DON_BY_BLOOD']['O+'] = 18
    params['MAX_DON_BY_BLOOD']['B-'] = 2
    params['MAX_DON_BY_BLOOD']['AB-'] = 3
    params['MAX_DON_BY_BLOOD']['A-'] = 6
    params['MAX_DON_BY_BLOOD']['O-'] = 7
    params['MAX_DON_BY_BLOOD']['A+'] = 14


    #The default weights to split the demand of a blood type is equal weights. The only requirement is that each
    #weight is positive and they add up to 1. 
    #Default
    params['SURGERYTYPES_PROP'] = {k:1/len(params['Surgerytypes']) for k in params['Surgerytypes']}
    params['SUBSTITUTION_PROP'] = {k:1/len(params['Substitution']) for k in params['Substitution']}

    # Set here the weights for each surgery type (if different than the default)
    params['SURGERYTYPES_PROP']['Urgent'] = 1/2
    params['SURGERYTYPES_PROP']['Elective'] = 1 - params['SURGERYTYPES_PROP']['Urgent']

    # Set here the weights for each substitution type (if different than the default)
    params['SUBSTITUTION_PROP'][True] = 1
    #params['SUBSTITUTION_PROP'][False] = 1 - params['SUBSTITUTION_PROP'][True]


    #Set here random surge parameters
    #params['TIME_PERIODS_SURGE'] = set([4,8,10,12,14])
    params['TIME_PERIODS_SURGE'] = set([3,6,10,13])
    #SURGE_PROB = 0.7
    params['SURGE_FACTOR'] = 6 #The surge demand is always going to be poisson with mean SURGE_FACTOR*params['MAX_DEM_BY_BLOOD'], even if the regular demand distribution is Uniform

    
    #Set here the weights for the utility function - urgent coverage, elective coverage, proportion of blood discarded
    params['WEIGHT_URGENT']= params['URGENT_DEMAND_BONUS']
    params['WEIGHT_ELECTIVE']= params['ELECTIVE_DEMAND_BONUS']
    params['WEIGHT_DISCARDED']= -1*params['DISCARD_BLOOD_PENALTY']

    # params['URGENT_DEMAND_BONUS'] = 10
    # params['ELECTIVE_DEMAND_BONUS'] = 1
    # params['DISCARD_BLOOD_PENALTY'] = -3 #applied for the oldest age in the holding/vfa arcs


    if (params['SAMPLING_DIST'] == 'P'):
        params['MAX_DEM_BY_BLOOD'] = {k: int(v * params['POISSON_FACTOR']) for k, v in params['MAX_DEM_BY_BLOOD'].items()}
        params['MAX_DON_BY_BLOOD'] = {k: int(v * params['POISSON_FACTOR']) for k, v in params['MAX_DON_BY_BLOOD'].items()}
        
        params['AVG_TOTAL_DEMAND'] = sum(params['MAX_DEM_BY_BLOOD'].values())
        params['AVG_TOTAL_SUPPLY'] = sum(params['MAX_DON_BY_BLOOD'].values())
        params['NUM_PARALLEL_LINKS'] = int(params['MAX_AGE']/2 * max(params['MAX_DON_BY_BLOOD'].values()))
        #print("Exogenous info dist: Poisson ")
    else:
        params['AVG_TOTAL_DEMAND'] = sum(params['MAX_DEM_BY_BLOOD'].values())/2
        params['AVG_TOTAL_SUPPLY'] = sum(params['MAX_DON_BY_BLOOD'].values())/2
        params['NUM_PARALLEL_LINKS'] = int(params['MAX_AGE']/2 * max(params['MAX_DON_BY_BLOOD'].values()))
        #print("Exogenous info dist: Uniform")

    #Checking if MYOPIC policy
    if not params['USE_VFA']:
        params['ALPHA'] = 0
        params['LOAD_VFA'] = False
        params['SAVE_VFA'] = False
        params['NUM_TRAINNING_ITER'] = 0
        params['NUM_ITER'] = params['NUM_TESTING_ITER']
        params['NUM_PARALLEL_LINKS'] = 1    


    # print("Printing params dict\n")
    # printParams(params)


    # if (params['SAMPLING_DIST'] == 'P'):
    #      print("Exogenous info dist: Poisson ")
    # else:
    #     print("Exogenous info dist: Uniform")
    
    # print("Demand parameters by blood type ",params['MAX_DEM_BY_BLOOD'])
    # print("There are ",params['NUM_SUR_TYPES'] * len(params['Substitution'])," demand nodes for each blood type")
    # print("Weights SURGERYTYPES_PROP ",params['SURGERYTYPES_PROP'])
    # print("Weights SUBSTITUTION_PROP ",params['SUBSTITUTION_PROP'])

    # print("Donation parameters by blood type ",params['MAX_DON_BY_BLOOD'])    

    # print("AVG TOTAL DEMAND ",params['AVG_TOTAL_DEMAND'])
    # print("AVG TOTAL SUPPLY ",params['AVG_TOTAL_SUPPLY'])
    # print("NUM PARALLEL LINKS ",params['NUM_PARALLEL_LINKS'])


    # print("Possible surge time periods ", params['TIME_PERIODS_SURGE'])
    # print("SURGE_PROB ", params['SURGE_PROB'], " and SURGE_FACTOR ", params['SURGE_FACTOR'])
        
        
    return params 





def initOutputListHeaders(params):
    labelsDemandExo=['Iteration','Time','Bloodtype','Urgency','isSubAllowed','DemandValue']
    labelsDonationExo=['Iteration','Time','Bloodtype','DonationValue']
    labelsSupplyPre=['Iteration','Time','BloodType','Age','PreInv']
    labelsSupplyPost=['Iteration','Time','BloodType','Age','PostInv']

    
    labelsSlopesList=['Iteration','Time','BloodType','Age']
    vNames = ["v_"+str(r) for r in list(range(params['NUM_PARALLEL_LINKS']))]
    labelsSlopesList = labelsSlopesList + vNames
   
    headerSolDemList =['Iteration',  'Time','BloodTypeS', 'Age','BloodTypeD', 'Urgency', 'SubsAllowed', 'isCompatible',  'Contrib', 'Value']    
    headerSolHoldList = ['Iteration','Time','BloodTypeS','Age','Value']  
    headerSimuList = ['Iteration','ElapsedTime','Stepsize','ObjVal','isTrainning']  
    headerUpdateVfaList = ['Iteration','Time','BloodType','Age','R','vhat','vbarOld','sqGrad','stepsize','vbarNew']
    
    return(labelsDemandExo, labelsDonationExo,labelsSupplyPre,labelsSupplyPost,labelsSlopesList,headerSolDemList,headerSolHoldList,headerSimuList,headerUpdateVfaList)


def convertToDfOutputLists(params,Bld_Net,demandExoList, donationExoList, supplyPreList, supplyPostList, slopesList, solDemList, solHoldList,  simuList, updateVfaList):
    labelsDemandExo, labelsDonationExo, labelsSupplyPre,  labelsSupplyPost, labelsSlopesList, headerSolDemList, headerSolHoldList, headerSimuList, headerUpdateVfaList = initOutputListHeaders(params)

    #Flatteting the lists 
    dfSimu = pd.DataFrame.from_records(simuList,columns=headerSimuList)

    demandExoListFlat = [(ite,t,dnode[0],dnode[1],dnode[2],dvalue) for ite,t,d in demandExoList for dnode,dvalue in zip(Bld_Net.demandnodes,d)]
    dfDemandExo = pd.DataFrame.from_records(demandExoListFlat,columns=labelsDemandExo)

    donationExoListFlat = [(ite,t,dtype,dvalue) for ite,t,d in donationExoList for dtype,dvalue in zip(params['Bloodtypes'],d)]
    dfDonationExo = pd.DataFrame.from_records(donationExoListFlat,columns=labelsDonationExo)

    supplyPreListFlat = [(ite,t,bnode[0],bnode[1],bvalue) for ite,t,b in supplyPreList for bnode,bvalue in zip(Bld_Net.bloodnodes,b)]
    dfSupplyPre = pd.DataFrame.from_records(supplyPreListFlat,columns=labelsSupplyPre)


    supplyPostListFlat = [(ite,t,bnode[0],bnode[1],bvalue) for ite,t,b in supplyPostList for bnode,bvalue in zip(Bld_Net.bloodnodes,b)]
    dfSupplyPost = pd.DataFrame.from_records(supplyPostListFlat,columns=labelsSupplyPost)


    solDemListFlat = [(ite,t,bld[0],bld[1],dem[0],dem[1],dem[2],params['SubMatrix'][(bld[0], dem[0])],Bld_Net.demweights[(bld,dem)],xbd) for ite,t,xDem in solDemList for bld,xb in zip(Bld_Net.bloodnodes,xDem) for dem,xbd in zip(Bld_Net.demandnodes,xb)]
    dfSolDem = pd.DataFrame.from_records(solDemListFlat,columns=headerSolDemList)

    solHoldListFlat = [(ite,t,bnode[0],bnode[1],hvalue) for ite,t,h in solHoldList for bnode,hvalue in zip(Bld_Net.bloodnodes,h)]
    dfSolHold = pd.DataFrame.from_records(solHoldListFlat,columns=headerSolHoldList)


    slopesListFlat = [(vnode[0],vnode[1],bnode[0],bnode[1],*list(vnode[2])) for vnode,bnode in zip(slopesList,Bld_Net.bloodnodes*params['NUM_ITER']*params['MAX_TIME'])]

    dfSlopes = pd.DataFrame.from_records(slopesListFlat,columns=labelsSlopesList)

    dfUpdateVfa = pd.DataFrame.from_records(updateVfaList,columns=headerUpdateVfaList)

    return dfDemandExo, dfDonationExo, dfSupplyPre, dfSupplyPost, dfSlopes, dfSolDem, dfSolHold,  dfSimu, dfUpdateVfa

def printDfsToOutputFile(params,dfDemandExo, dfDonationExo, dfSupplyPre, dfSupplyPost, dfSlopes, dfSolDem, dfSolHold,  dfSimu, dfUpdateVfa):

    t_init_print = time.time()
    print("Started printing file")
    # print to excel file
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(params['OUTPUT_FILENAME'], engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object.
    dfSimu.to_excel(writer, sheet_name='Simu')
    
    if params['PRINT_ALL']:
        dfDemandExo.to_excel(writer, sheet_name='DemandExo')
        dfDonationExo.to_excel(writer, sheet_name='DonationExo')
        dfSupplyPre.to_excel(writer, sheet_name='SupplyPre')
        dfSolDem.to_excel(writer, sheet_name='SolDem')
        dfSolHold.to_excel(writer, sheet_name='HoldDem')
        dfSupplyPost.to_excel(writer, sheet_name='SupplyPost')
        dfSlopes.to_excel(writer, sheet_name='SlopesList')
        dfUpdateVfa.to_excel(writer, sheet_name='UpdatesVfa')


    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

    print("Finished printing files in {:.2f} secs".format(time.time()-t_init_print))
  


In [38]:
def Main(param_dict):    
    
    t_global_init = time.time()
    print("********************Started Main*****************\n")
    # params = loadParams('Parameters.xlsx')
    params = loadParams(param_dict)
    # print(params)
    alpha = params['ALPHA']
    
    #ite_TRA=np.arange(0, params['NUM_TRAINNING_ITER'], 1)
    #selectedIte = list(set([0,5,10,19]) & set(ite_TRA))
    
    

    # initializing the random seed for trainning iterations
    np.random.seed(params['SEED_TRAINING'])

    # initializing the blood network
    Bld_Net = create_bld_net(params)

    if (params['LOAD_VFA'] and os.path.exists(params['NAME_LOAD_VFA_PICKLE'])):
        pickle_off = open(params['NAME_LOAD_VFA_PICKLE'],"rb")
        Other_Bld_Net = pickle.load(pickle_off)
        Bld_Net.varr = Other_Bld_Net.varr.copy()
        Bld_Net.parallelarr = Other_Bld_Net.parallelarr.copy()

    # initializing the model
    state_names = ['BloodInventory', 'Demand', 'Donation']
    decision_names = ['Hold', 'Contribution']  

    # initializing the lists that will store the all the info/decisions/states/slopes along the iterations for printing purposes
    demandExoList, donationExoList, supplyPreList, supplyPostList, slopesList, solDemList, solHoldList,  simuList, updateVfaList = [],[],[],[],[],[],[],[],[]

    #initializing the policy
    P = Policy(params,Bld_Net)

    
    iteration = 0
    obj = []

    # if (params['NUM_TRAINNING_ITER']>0):
    #     print("\n Starting training iterations\n")

    while iteration < params['NUM_ITER']:  
        IS_TRAINING = (iteration<params['NUM_TRAINNING_ITER'])
        if (iteration==params['NUM_TRAINNING_ITER']):
            # print("Starting testing iterations! Currently at iteration ",iteration)
            # print("Reseting random seed!")
            np.random.seed(params['SEED_TESTING'])
            
        # t_init = time.clock()
        t_init = time.process_time()
        # print('Iteration = ', iteration)
        
        # Initial inventory
        if (params['SAMPLING_DIST'] == 'P'):
            bldinv_init = [int(np.random.poisson(params['MAX_DON_BY_BLOOD'][bld[0]])*.9) if bld[1]=='0' else int(np.random.poisson(params['MAX_DON_BY_BLOOD'][bld[0]])*(0.1/(params['MAX_AGE']-1))) for bld in Bld_Net.bloodnodes]
        else:
            bldinv_init = [round(np.random.uniform(0, params['MAX_DON_BY_BLOOD'][bld[0]])*.9) if bld[1]=='0' else round(np.random.uniform(0, params['MAX_DON_BY_BLOOD'][bld[0]])*(0.1/(params['MAX_AGE']-1))) for bld in Bld_Net.bloodnodes]
    
        # Initial exogenous information
        if (params['SAMPLING_DIST'] == 'P'):
            exog_info_init = generate_exog_info_by_bloodtype_p(0, Bld_Net, params)
        else:
            exog_info_init = generate_exog_info_by_bloodtype(0, Bld_Net, params)

        #initial state - the donation is irrelevant at time period zero - only the initial invetory counts
        init_state = {'BloodInventory': bldinv_init, 'Demand': exog_info_init.demand, 'Donation' : exog_info_init.donation}

        M = Model(state_names, decision_names, init_state, Bld_Net,params)
        #print("Initial blood supply across {} types and {} ages is {}".format(params['NUM_BLD_TYPES'],params['MAX_AGE'],sum(M.bld_inv)))
        #print("Initial demand across {} types and {} urgency states and {} substitution states is {}".format(params['NUM_BLD_TYPES'],params['NUM_SUR_TYPES'],len(params['Substitution']),sum(M.demand)))

        
        t = 0        
        obj.append(0)

        #Steping forward in time
        while t < params['MAX_TIME']:
          
            
            #Compute the solution for time period t - return the solution, the value, the dual and the updated lists
            sol,val,x,hld,d,solDemList,solHoldList=P.getLPSol(params,M,iteration,t,solDemList,solHoldList,IS_TRAINING)
            obj[iteration] += val
            
            
            #Grabbing exogenous data to construct data frame
            recordDemandExo = (iteration,t,M.Bld_Net.demandamount.copy())
            demandExoList.append(recordDemandExo)       
            if (t==0):   
                recordDonationExo = (iteration,0,list(np.array(M.bld_inv)[::params['MAX_AGE']]))
                donationExoList.append(recordDonationExo)
            
            if (t<params['MAX_TIME']-1):
                recordDonationExo = (iteration,t+1,M.donation.copy())
                donationExoList.append(recordDonationExo)
                
            #Grabbing pre-decision state to construct data frame
            recordSupplyPre = (iteration,t,M.bld_inv.copy())
            supplyPreList.append(recordSupplyPre)
                
            
            if IS_TRAINING:    
                alpha,slopesList,updateVfaList = P.updateVFAs(params,M,iteration,t,d, slopesList,updateVfaList)
                            
            # build decision
            dcsn = M.build_decision({'Hold': hld, 'Contribution': val})
                    
            M.transition_fn(dcsn)
            
            #Grabbing post-decision state to construct data frame
            recordSupplyPost = (iteration,t,M.bld_inv.copy())
            supplyPostList.append(recordSupplyPost)
            
            t += 1
            # generate/read exogenous information 
            if (params['SAMPLING_DIST'] == 'P'):
                exog_info = generate_exog_info_by_bloodtype_p(t, Bld_Net, params)
            else:
                exog_info = generate_exog_info_by_bloodtype(t, Bld_Net, params)
            M.exog_info_fn(exog_info)
            
        
        
        # copy v to the parallel links
        for t in params['Times']:
            for hld in M.Bld_Net.holdnodes:
                parArr = 1 * M.Bld_Net.varr[(t,hld, M.Bld_Net.supersink)]
                M.Bld_Net.add_parallel(t,hld, M.Bld_Net.supersink, parArr)
        
          

        # t_end = time.clock()
        t_end = time.process_time()
        recordSimu = (iteration,int(t_end-t_init),alpha,obj[iteration],(iteration<params['NUM_TRAINNING_ITER']))
        simuList.append(recordSimu)
       
        # print("***Finishing iteration {} in {:.2f} secs. Total contribution: {:.2f}***\n".format(recordSimu[0],recordSimu[1],recordSimu[3]))
        
        
        iteration += 1
        
    #End of iterations
    ###########################################################################################################################################


    if (params['SAVE_VFA']):
        pickling_on = open(params['NAME_SAVE_VFA_PICKLE'],"wb")
        pickle.dump(M.Bld_Net, pickling_on)
        pickling_on.close()

    

    ###########################################################################################################################################
    #Computing stats and plots
    ###########################################################################################################################################
    dfDemandExo, dfDonationExo, dfSupplyPre, dfSupplyPost, dfSlopes, dfSolDem, dfSolHold,  dfSimu, dfUpdateVfa = convertToDfOutputLists(params,Bld_Net,demandExoList, donationExoList, supplyPreList, supplyPostList, slopesList, solDemList, solHoldList,  simuList, updateVfaList)

    # print(dfDemandExo)

    policy = params['USE_VFA'] and 'VFA-Based' or 'MYOPIC'
    surge  = params['SURGE_PROB']>0 and "SURGE_"+str(params['SURGE_PROB']) or "NO_SURGE"
    instance = "Policy{}_{}_PEN_{:,}_ALPHA_{:.2f}".format(policy,surge,params['BLOOD_FOR_ELECTIVE_PENALTY'],params['ALPHA'])

    #Average Contribution
    meanTesting = dfSimu.groupby('isTrainning')['ObjVal'].mean()[False]
    stdevTesting = dfSimu.groupby('isTrainning')['ObjVal'].std(ddof = 0)[False]

    # print(dfSimu.groupby('isTrainning')['ObjVal'])
    # print(stdevTesting)
    #Total Blood discarded
    totalDiscarded = dfSolHold.loc[(dfSolHold.Age.astype(int)>params['MAX_AGE']-2)&(dfSolHold.Iteration>=params['NUM_TRAINNING_ITER']),:].copy()['Value'].sum()

    #Total Donation
    totalDonation = dfDonationExo[dfDonationExo.Iteration>=params['NUM_TRAINNING_ITER']].copy()['DonationValue'].sum()

    #Coverage 
    dfCoverage = dfSolDem.groupby(['BloodTypeD', 'Urgency','Iteration',  'Time'])['Value'].sum()
    dfCoverage.index = dfCoverage.index.rename("Bloodtype", level=0)   
    dfCoverage = pd.concat([dfCoverage,dfDemandExo.groupby(['Bloodtype', 'Urgency','Iteration',  'Time'])['DemandValue'].sum()],axis=1)
    dfCoverage['Ratio']= dfCoverage['Value']/ dfCoverage['DemandValue']
    
    
    dfCoverage_agg_ite = dfCoverage.groupby(['Bloodtype', 'Urgency','Iteration'])['Ratio'].mean().reset_index()

    numTra = params['NUM_TRAINNING_ITER']
    dfCoverage_agg_test = dfCoverage_agg_ite.query('Iteration >= @numTra')
    dfPrintIte=dfCoverage_agg_test.pivot_table('Ratio',index='Bloodtype',columns='Urgency')

    

    finalCoverage=dfCoverage_agg_test.groupby('Urgency')['Ratio'].mean()

    coverage = "Average Coverage: -  Urgent: {:.2f} Elective: {:.2f} Avg: {:.2f}".format(finalCoverage['Urgent'],finalCoverage['Elective'],dfCoverage_agg_ite['Ratio'].mean())
    
    #dfUtility = dfCoverage.query('Iteration >= @numTra').copy().reset_index()
    #dfUtility['Weight']=-1
    #dfUtility['Score']=0
    #dfUtility.loc[dfUtility.Urgency=="Elective",'Weight']=1
    #dfUtility.loc[dfUtility.Urgency=="Urgent",'Weight']=100
    #dfUtility.loc[(dfUtility.Urgency=="Urgent") & (dfUtility.Ratio>.9),'Score']=1
    #sumRatio=(dfUtility['Ratio']*dfUtility['Weight']).sum()
    #sumWeight=(dfUtility['Weight']).sum()
    #sumScore=(dfUtility['Score']).sum()
    #utility=sumRatio/sumWeight
    
    # print('final_cov',finalCoverage)
    # print('discard', totalDiscarded)
    # print('donat', totalDonation)
    #Utility function
    # utility=(params['WEIGHT_URGENT']*round(finalCoverage['Urgent'],2)+params['WEIGHT_ELECTIVE']*round(finalCoverage['Elective'],2))*100
    # modifiedUtil=utility-params['WEIGHT_DISCARDED']*100*round(totalDiscarded/totalDonation,2)

    idx = pd.IndexSlice
    ite_TES =  np.arange(0, params['NUM_TESTING_ITER'], 1) + params['NUM_TRAINNING_ITER']
    dfCoverage=dfCoverage[dfCoverage.DemandValue>0].copy()
    utility_lst = []
    
    for i in ite_TES:
        ite_TES_i = np.array([i,])
        # print(ite_TES_i)
        uncoverU = dfCoverage.loc[idx[:,['Urgent'],ite_TES_i,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,['Urgent'],ite_TES_i,:],['Value']].values.sum() 
        uncoverE = dfCoverage.loc[idx[:,['Elective'],ite_TES_i,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,['Elective'],ite_TES_i,:],['Value']].values.sum() 
        uncoverT = dfCoverage.loc[idx[:,:,ite_TES_i,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,:,ite_TES_i,:],['Value']].values.sum() 

        demandU = dfCoverage.loc[idx[:,['Urgent'],ite_TES_i,:],['DemandValue']].values.sum() 
        demandE = dfCoverage.loc[idx[:,['Elective'],ite_TES_i,:],['DemandValue']].values.sum() 
        demandT = dfCoverage.loc[idx[:,:,ite_TES_i,:],['DemandValue']].values.sum() 


        discard_i = dfSolHold.loc[(dfSolHold.Age.astype(int)>params['MAX_AGE']-2)&(dfSolHold.Iteration==i),:].copy()['Value'].sum()
        utility=params['WEIGHT_URGENT']*(demandU-uncoverU)+params['WEIGHT_ELECTIVE']*(demandE-uncoverE)
        modifiedUtil=utility-(params['WEIGHT_DISCARDED']*discard_i)
        utility_lst.append(modifiedUtil)
    # print(utility_lst)
    # print(np.mean(utility_lst))




    #####3



    # dfCoverage=dfCoverage[dfCoverage.DemandValue>0].copy()
    # uncoverU = dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['Value']].values.sum() 
    # uncoverE = dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['Value']].values.sum() 
    # uncoverT = dfCoverage.loc[idx[:,:,ite_TES,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,:,ite_TES,:],['Value']].values.sum() 

    # demandU = dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['DemandValue']].values.sum() 
    # demandE = dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['DemandValue']].values.sum() 
    # demandT = dfCoverage.loc[idx[:,:,ite_TES,:],['DemandValue']].values.sum() 

    # utility=(params['WEIGHT_URGENT']*(demandU-uncoverU)+params['WEIGHT_ELECTIVE']*(demandE-uncoverE))/params['NUM_TESTING_ITER']
    # modifiedUtil=utility-(params['WEIGHT_DISCARDED']*totalDiscarded)/params['NUM_TESTING_ITER']
    # print(modifiedUtil)

    # print(sum(uncoverU_lst), sum(uncoverE_lst), sum(uncoverT_lst))
    # print(uncoverU, uncoverE, uncoverT)
    # print(sum(demandU_lst), sum(demandE_lst), sum(demandT_lst))
    # print(demandU, demandE, demandT)   


    # print(f"wefwe {demandT}")
    # print(f"Average Total Demand (urgent): {demandU/params['NUM_TESTING_ITER']:.2f}")
    # print(f"Average Total Demand Uncovered (urgent): {uncoverU/params['NUM_TESTING_ITER']:.2f}"
    # print(f"Average Total Demand (elective): {demandE/params['NUM_TESTING_ITER']:.2f}")
    # print(f"Average Total Demand Uncovered (elective): {uncoverE/params['NUM_TESTING_ITER']:.2f}"
    # print(f"Average units of blood discarded: {totalDiscarded/params['NUM_TESTING_ITER']:.2f}")
    # print('demandU', demandU)
    # print('demandE', demandE)
    # print('totalDis', totalDiscarded)

    if params['SHOW_PLOTS'] == True:

        
        ###########################################################################################################################################
        #Figure 1 - Total Contribution along iterations
        ite = np.arange(0, params['NUM_ITER'], 1)
        ite_TRA =  np.arange(0, params['NUM_TRAINNING_ITER'], 1)
        ite_TES =  np.arange(0, params['NUM_TESTING_ITER'], 1) + params['NUM_TRAINNING_ITER']
        
        fig_ite, ax_ite = plt.subplots(figsize=(16,8))
        ax_ite.plot(ite,dfSimu['ObjVal'],'g-',label='_nolegend_')
        ax_ite.plot(ite_TRA,dfSimu['ObjVal'][ite_TRA],'g-',label="Training",marker='o')
        ax_ite.plot(ite_TES,dfSimu['ObjVal'][ite_TES],'b-',label="Testing",marker='o')
        ax_ite.hlines(meanTesting, ite_TES[0], ite_TES[-1], color='b',linestyle='--',label="Avg Testing",linewidth=4)
        
        ax_ite.axvline(ite_TES[0], 0, 1, color='k',linestyle=':')
        ax_ite.legend()
        ax_ite.set_xlabel('Iterations',fontsize=12)
        ax_ite.set_ylabel('$',fontsize=12)
        #ax_ite.set_ylim([20000,34000])
        ax_ite.set_title("Figure 1: Policy {}_{} - Total contributions \n Avg total contribution during TESTING iterations: ${:,}\n Final utility: {:.2f}".format(policy,surge,meanTesting,modifiedUtil))
        ax_ite.set_xticks(ite)
        ax_ite.set_xticklabels(list(ite_TRA)+list(np.arange(0, params['NUM_TESTING_ITER'], 1)))
        for c in ite_TES:
            ax_ite.get_xticklabels()[c].set_color("b")
        ###########################################################################################################################################

        
        ###########################################################################################################################################
        #Figure 2 - Exogenous processes - Demand and Donation
        fig_exo, ax_exo = plt.subplots(2,1,figsize=(16,8),sharex=True)
        dfDemandExoP = dfDemandExo[dfDemandExo.Iteration>=params['NUM_TRAINNING_ITER']].copy()
        dfDemandExoP['Iteration'] = dfDemandExoP['Iteration'] - params['NUM_TRAINNING_ITER']
        dfPrintDemand = dfDemandExoP.pivot_table('DemandValue',index='Time',columns='Iteration',aggfunc='sum')

        # print(dfDemandExoP.pivot_table('DemandValue',index='Time',columns='Iteration',aggfunc='sum'))

        
        l=dfPrintDemand.plot(ax=ax_exo[0],title="Total Demand - {}".format(surge),legend=False)

        dfDonationExoP = dfDonationExo[dfDonationExo.Iteration>=params['NUM_TRAINNING_ITER']].copy()
        dfDonationExoP['Iteration'] = dfDonationExoP['Iteration'] - params['NUM_TRAINNING_ITER']
        dfPrintDonation = dfDonationExoP.pivot_table('DonationValue',index='Time',columns='Iteration',aggfunc='sum')
        dfPrintDonation.plot(ax=ax_exo[1],title="Total Donation",legend=False)
        ax_exo[1].set_xlabel("Time period",fontsize=12)
        ax_exo[0].set_ylabel("Units",fontsize=12)
        ax_exo[1].set_ylabel("Units",fontsize=12)
        
        fig_exo.legend(l,labels=list(np.arange(0, params['NUM_TESTING_ITER'], 1)),title="Iteration",loc="center right",fancybox=True, shadow=True)
        fig_exo.suptitle("Figure 2 - Exogenous processes along the testing iterations")
        ###########################################################################################################################################

        
        
        dfInv = dfSupplyPre[dfSupplyPre.Iteration>=params['NUM_TRAINNING_ITER']].copy()
        dfInv['Iteration'] = dfInv['Iteration'] - params['NUM_TRAINNING_ITER']
        
        ###########################################################################################################################################
        #Figure 3 - Pre decision inventory levels by age
        fig_inv, ax_inv = plt.subplots(3,1,figsize=(16,10),sharex=True)
        for age in [0,1,2]:

            strage=str(age)

            dfPrint = dfInv[dfInv.Age == strage].pivot_table('PreInv',index=['Time'],columns=['Iteration'],aggfunc='sum')
            l=dfPrint.plot(ax=ax_inv[age],legend=False)


            dfPrintAvg = dfInv[dfInv.Age == strage].groupby(['Time','Iteration'])['PreInv'].sum()
            dfPrintAvg = dfPrintAvg.groupby('Time').mean()
            avg_line = ax_inv[age].plot(dfPrintAvg.index,dfPrintAvg.values,'k',linestyle=':',marker='s',markersize='12',label='Average')
            ax_inv[age].set_title("Age: {} - Avg inventory level: {:.0f}".format(age,dfPrintAvg.values.mean()))
            ax_inv[age].set_ylabel("Units",fontsize=12)

        ax_inv[2].set_xlabel("Time period",fontsize=12)
        fig_inv.suptitle("Figure 3 - Policy {}_{} \n Pre-decision  inventory level all blood types".format(policy,surge))
        fig_inv.legend([l,avg_line],labels=list(np.arange(0, params['NUM_TESTING_ITER'], 1))+["Avg"],title="Iteration",loc='center right',fancybox=True, shadow=True, ncol=1)
        ###########################################################################################################################################


        ###########################################################################################################################################
        #Figure 4 - Pre decision inventory levels by bloodtype
        fig_inv_blood, ax_inv_blood = plt.subplots(4,2,figsize=(16,10),sharex=True)
        row = -1
        for m,b in enumerate(params['Bloodtypes']):

            col= (m)%2
            if col == 0:
                row+=1
                ax_inv_blood[row,col].set_ylabel("Units",fontsize=8)

            dfPrint = dfInv[dfInv.BloodType == b].pivot_table('PreInv',index=['Time'],columns=['Iteration'],aggfunc='sum')
            l=dfPrint.plot(ax=ax_inv_blood[row,col],legend=False)

            dfPrintAvg = dfInv[dfInv.BloodType == b].groupby(['Time','Iteration'])['PreInv'].sum()
            dfPrintAvg = dfPrintAvg.groupby('Time').mean()

            avg_line =  ax_inv_blood[row,col].plot(dfPrintAvg.index,dfPrintAvg.values,'k',linestyle=':',marker='s',markersize='10',label='Average')

            ax_inv_blood[row,col].set_title("Bloodtype: {}  - Avg inventory level: {:.0f}".format(b,dfPrintAvg.values.mean()),fontsize=8)
            ax_inv_blood[row,col].set_xlabel("Time Period",fontsize=8)

        fig_inv_blood.suptitle("Figure 4 - Policy {}_{} \n Pre-decision  inventory level all ages".format(policy,surge))
        fig_inv_blood.legend([l,avg_line],labels=list(np.arange(0, params['NUM_TESTING_ITER'], 1))+["Avg"],loc="center right", title="Iteration")
        ###########################################################################################################################################


        ###########################################################################################################################################
        #Figure 5 - Pre decision inventory levels 
        dfPrintAvg = dfInv.groupby(['Time','Iteration'])['PreInv'].sum()
        dfPrintAvg = dfPrintAvg.groupby('Time').mean()
        fig_inv_total, ax_inv_total = plt.subplots(figsize=(16,8))
        dfPrint = dfInv.pivot_table('PreInv',index=['Time'],columns=['Iteration'],aggfunc='sum')
        dfPrint.plot(ax=ax_inv_total,legend=True)
        first_legend = ax_inv_total.legend(title="Iteration")
        avg_line = ax_inv_total.plot(dfPrintAvg.index,dfPrintAvg.values,'k',linestyle=':',marker='s',markersize='12',label='Average')
        ax_inv_total.legend(title="Iteration")
        ax_inv_total.set_xlabel("Time period")
        ax_inv_total.set_ylabel("Units")
        fig_inv_total.suptitle("Figure 5 - Policy {}_{} \n Pre-decision inventory level all blood types and ages \n Average inventory level across time periods: {:.0f}".format(policy,surge,dfPrintAvg.values.mean()))
        ###########################################################################################################################################


        
        

        ###########################################################################################################################################
        #Figure 6 - Discarded blood during testing iterations
        dfDiscarded = dfSolHold.loc[(dfSolHold.Age.astype(int)>params['MAX_AGE']-2)&(dfSolHold.Iteration>=params['NUM_TRAINNING_ITER']),:].copy()
        dfDiscarded = dfDiscarded.groupby(['BloodTypeS','Time'])['Value'].sum().reset_index()
        dfDiscarded['Prop']=100*dfDiscarded['Value']/totalDonation
        

        y_a = dfDiscarded.BloodTypeS.unique()
        x_a = dfDiscarded.Time.unique()
        discarded_matrix = np.reshape(np.array(dfDiscarded['Value']), (-1, len(x_a)))
    
        
        fig_dis, ax_dis = plt.subplots(figsize=(16,8))  
        im = ax_dis.imshow(discarded_matrix, cmap='hot_r',origin='lower',aspect='auto',alpha=.9)
        cbar = ax_dis.figure.colorbar(im, ax=ax_dis,label='Units of blood')  
        ax_dis.set_xticks(np.arange(len(x_a)))
        ax_dis.set_yticks(np.arange(len(y_a)))
        ax_dis.set_xticklabels(x_a)
        ax_dis.set_yticklabels(y_a)
        ax_dis.set_xlabel("Time Period")
        ax_dis.set_title("Figure 6 - Policy {}_{} \n Total discarded blood during TESTING iterations\n Proportion of blood discarded: {:.2f}%".format(policy,surge,totalDiscarded*100/totalDonation))

        ###########################################################################################################################################


        ###########################################################################################################################################
        #Figure 7 - Demand Coverage - Testing iterations
        fig_tes, ax_tes = plt.subplots(figsize=(16,8))   
        ax_tes.plot(dfPrintIte['Urgent'],marker='o', label = 'Urgent')
        ax_tes.plot(dfPrintIte['Elective'],marker='o', label = 'Elective')
        ax_tes.set_title("Figure 7 -Policy {}_{} \n Average coverage of demand by blood type and urgency level during TESTING iterations\n {} Coverage utility: {:.0f}".format(policy,surge,coverage,utility))
        ax_tes.set_ylabel("Coverage ratio")
        ax_tes.legend(title="Urgency level",loc='center left', bbox_to_anchor=(1, 0.5),fancybox=True, shadow=True, ncol=1)
        ###########################################################################################################################################


        ###########################################################################################################################################
        #Figure 8 - Demand Coverage - Along Blood types
        iteS = np.arange(0, params['NUM_ITER'], 1)
        if True:
            
            #selectedIte = sorted(list(set([0,5,10,19]) ))
            selectedIte = sorted(list(set([0,5,10,19]) & set(iteS)))
            fig_cover, ax_cover = plt.subplots(1,len(selectedIte),figsize=(16,8),sharey=True)

            

            for i,ite in enumerate(selectedIte):
                dfPrintIte = dfCoverage_agg_ite[dfCoverage_agg_ite.Iteration==ite]
                dfPrintIte=dfPrintIte.pivot_table('Ratio',index='Bloodtype',columns='Urgency')

                typIte='TES'
                if ite <params['NUM_TRAINNING_ITER']:
                    typIte='TRA'

                
                if len(selectedIte)==1:
                    ax_cover.plot(dfPrintIte['Urgent'],marker='o', label = 'Urgent')
                    ax_cover.plot(dfPrintIte['Elective'],marker='o', label = 'Elective')
                    ax_cover.set_title("Iteration {} ({})".format(ite,typIte))
                else:
                    ax_cover[i].plot(dfPrintIte['Urgent'],marker='o', label = 'Urgent')
                    ax_cover[i].plot(dfPrintIte['Elective'],marker='o', label = 'Elective')
                    ax_cover[i].set_title("Iteration {} ({})".format(ite,typIte))
            
                fig_cover.suptitle("Figure 8 - Policy {}-{} \n Average coverage of demand by blood type and urgency level for different iterations".format(policy,surge))

            if len(selectedIte)==1:
                ax_cover.set_ylabel("Coverage ratio")
                ax_cover.legend(title="Urgency level",loc='center left', bbox_to_anchor=(1, 0.5),fancybox=True, shadow=True, ncol=1)
            else:
                ax_cover[0].set_ylabel("Coverage ratio")
                ax_cover[len(selectedIte)-1].legend(title="Urgency level",loc='center left', bbox_to_anchor=(1, 0.5),fancybox=True, shadow=True, ncol=1)    
        ###########################################################################################################################################


        ###########################################################################################################################################
        #Figure 9 - Demand Coverage - Along time periods
        if True:
            dfCoverage_agg_ite = dfCoverage.groupby(['Time', 'Urgency','Iteration'])['Ratio'].mean().reset_index()
            #selectedIte = sorted(list(set([0,5,10,19]) ))
            selectedIte = sorted(list(set([0,5,10,19]) & set(iteS)))
            fig_cover_ite, ax_cover_ite = plt.subplots(1,len(selectedIte),figsize=(16,8),sharey=True,sharex=True)
            for i,ite in enumerate(selectedIte):
                dfPrintIte = dfCoverage_agg_ite[dfCoverage_agg_ite.Iteration==ite]
                dfPrintIte=dfPrintIte.pivot_table('Ratio',index='Time',columns='Urgency')

                typIte='TES'
                if ite <params['NUM_TRAINNING_ITER']:
                    typIte='TRA'
                
                if len(selectedIte)==1:
                    ax_cover_ite.plot(dfPrintIte['Urgent'],marker='o', label = 'Urgent')
                    ax_cover_ite.plot(dfPrintIte['Elective'],marker='o', label = 'Elective')
                    ax_cover_ite.set_title("Iteration {} ({})".format(ite,typIte))
                    ax_cover_ite.set_xticks(list(range(0,params['MAX_TIME'],2)))
                    ax_cover_ite.set_xticklabels(list(range(0,params['MAX_TIME'],2)))
                    ax_cover_ite.set_xlabel("Time Period")
                else:
                    ax_cover_ite[i].plot(dfPrintIte['Urgent'],marker='o', label = 'Urgent')
                    ax_cover_ite[i].plot(dfPrintIte['Elective'],marker='o', label = 'Elective')
                    ax_cover_ite[i].set_title("Iteration {} ({})".format(ite,typIte))
                    ax_cover_ite[i].set_xticks(list(range(0,params['MAX_TIME'],2)))
                    ax_cover_ite[i].set_xticklabels(list(range(0,params['MAX_TIME'],2)))
                    ax_cover_ite[i].set_xlabel("Time Period")

                fig_cover_ite.suptitle("Figure 9 - Policy {}-{} \n Average coverage of demand by time period and urgency level for different iterations".format(policy,surge))

            if len(selectedIte)==1:
                ax_cover_ite.set_ylabel("Coverage ratio")
                ax_cover_ite.legend(title="Urgency level",loc='center left', bbox_to_anchor=(1, 0.5),fancybox=True, shadow=True, ncol=1)
            else:
                ax_cover_ite[0].set_ylabel("Coverage ratio")
                ax_cover_ite[len(selectedIte)-1].legend(title="Urgency level",loc='center left', bbox_to_anchor=(1, 0.5),fancybox=True, shadow=True, ncol=1)


        ###########################################################################################################################################
        #Figure 10 - Histogram Demand Coverage - Along time periods       
        idx = pd.IndexSlice
        dfCoverage=dfCoverage[dfCoverage.DemandValue>0].copy()

        uncoverU = dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['Value']].values.sum() 
        uncoverE = dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['Value']].values.sum() 
        uncoverT = dfCoverage.loc[idx[:,:,ite_TES,:],['DemandValue']].values.sum() - dfCoverage.loc[idx[:,:,ite_TES,:],['Value']].values.sum() 

        demandU = dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['DemandValue']].values.sum() 
        demandE = dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['DemandValue']].values.sum() 
        demandT = dfCoverage.loc[idx[:,:,ite_TES,:],['DemandValue']].values.sum() 
        

        fig_hist, ax_hist = plt.subplots(1,3,figsize=(16,8),sharey=True,sharex=True)
        ax_hist[0].hist(dfCoverage.loc[idx[:,['Urgent'],ite_TES,:],['Ratio']].values, bins=11,color='tab:blue')
        ax_hist[0].set_title("Urgent")
        ax_hist[0].set_ylabel("Count")
        ax_hist[0].set_xlim([0,1])
        ax_hist[0].annotate('Uncovered Demand: {:,}\n Total Demand: {:,}'.format(int(uncoverU),demandU),xy=(.8, .975), xycoords='axes fraction',horizontalalignment='right', verticalalignment='top',fontsize=12)
        
        ax_hist[1].hist(dfCoverage.loc[idx[:,['Elective'],ite_TES,:],['Ratio']].values, bins=11,color='tab:orange')
        ax_hist[1].set_title("Elective")
        ax_hist[1].set_xlabel("Coverage Ratio")
        ax_hist[1].annotate('Uncovered Demand: {:,}\n Total Demand: {:,}'.format(int(uncoverE),demandE),xy=(.8, .975), xycoords='axes fraction',horizontalalignment='right', verticalalignment='top',fontsize=12)


        ax_hist[2].hist(dfCoverage.loc[idx[:,:,ite_TES,:],['Ratio']].values, bins=11,color='tab:gray')
        ax_hist[2].set_title("Total")
        ax_hist[2].annotate('Uncovered Demand: {:,}\n Total Demand: {:,}'.format(int(uncoverT),demandT),xy=(.8, .975), xycoords='axes fraction',horizontalalignment='right', verticalalignment='top',fontsize=12)

        fig_hist.suptitle("Figure 10 - Policy {}-{} \n Histogram of blood coverage - All TESTING iterations and time periods\n {} Coverage utility: {:.0f}".format(policy,surge,coverage,utility))
        ###########################################################################################################################################
        
        if params['SAVE_PLOTS']:
            fig_ite.savefig('{}_Figure1.pdf'.format(instance))
            fig_exo.savefig('{}_Figure2.pdf'.format(instance))
            fig_inv.savefig('{}_Figure3.pdf'.format(instance))
            fig_inv_blood.savefig('{}_Figure4.pdf'.format(instance))
            fig_inv_total.savefig('{}_Figure5.pdf'.format(instance))
            fig_dis.savefig('{}_Figure6.pdf'.format(instance))
            fig_tes.savefig('{}_Figure7.pdf'.format(instance))
            fig_cover.savefig('{}_Figure8.pdf'.format(instance))
            fig_cover_ite.savefig('{}_Figure9.pdf'.format(instance))
            fig_hist.savefig('{}_Figure10.pdf'.format(instance))

        if params['SHOW_PLOTS']:
            plt.show()

        ###########################################################################################################################################
    # Printing the final results


    res_dict = dict()

    # print("\n*******************************************************************************************")
    res_dict["Policy: "] = (policy,surge)
    res_dict['name'] = instance
    res_dict["coverage"] = coverage
    res_dict["Percentage of blood discarded: "] = totalDiscarded*100/totalDonation
    res_dict["Average Total Demand (urgent): "] = demandU/params['NUM_TESTING_ITER']
    res_dict["Average Total Demand Uncovered (urgent): "] = uncoverU/params['NUM_TESTING_ITER']
    res_dict["Average Total Demand (elective): "] = demandE/params['NUM_TESTING_ITER']
    res_dict["Average Total Demand Uncovered (elective): "] = uncoverE/params['NUM_TESTING_ITER']
    res_dict["Average units of blood discarded: "]=totalDiscarded/params['NUM_TESTING_ITER']
    res_dict["Average total contribution during TESTING iterations"] = meanTesting
    res_dict["StDev total contribution during TESTING iterations"] = stdevTesting
    res_dict["Average utility: "] = np.mean(utility_lst)
    res_dict["StDev utility"] = np.std(utility_lst)
    
    return res_dict

## Utility Function
This is the objective function we are trying to optimise (in expectation). It is calculated for a particular traning run as
$$
\text{Utility} = c^{urgent} \times cov_u + c^{elective} \times cov_e + c^{discard} \times discard,
$$
where $cov_u$ is the number of urgent demands covered over the time horizon, $cov_e$ is the number of elective demands covered over the time horizon and $discard$ is the number of units of blood discarded over the time horizon.

$c^{urgent}>0$, $c^{elective} >0$ and $c^{discard}<0$ are inputs to the problem.

Utility will be reported as an averaged over the number of testing iterations.

## Figure Descriptions

- Figure 1: Total Contribution along iterations, When using a VFA-based policy, we will plot both training and testing iterations. When using a Myopic policy, we will only plot the testing iterations.
- Figure 2: Exogenous processes  during testing iterations, Total Demand and Donation along time periods.
- Figure 3: predecision inventory levels during testing iterations by blood age along time periods
- Figure 4: predecision inventory levels during testing iterations by blood type along time periods
- Figure 5: Total predecision inventory levels		
- Figure 6: Discarded blood during testing iterations		
- Figure 7: Demand coverage during testing iterations along blood types		
- Figure 8: Demand coverage for selected iterations, along lood types 		
- Figure 9: Demand coverage for selected  iterations, along time periods		
- Figure 10: Histogram of coverage ratios during testing iterations		

## Parameter Descriptions:

- `BLOOD_FOR_ELECTIVE_PENALTY`:	Penalty for covering elective demand. Used to guide the myopic policy.	
- `SURGE_PROB`:	Probability that a surge in demand might happen, $p^{surge}$.
- `NUM_TRAINNING_ITER`:	Number of trainning iterations, $n^{train}$. Used in the VFA-based policy (`USE_VFA` = `True`). It is going to be ignored when `USE_VFA` is set to `False`	
- `NUM_TESTING_ITER`: Number of testing iterations, $n^{test}$.
- `USE_VFA`: Parameter to indicate if  we should use a VFA-based policy. Set it to `True` if you want to use a VFA and set it to `False` if you want a Myopic policy.	
- `ALPHA`: The stepsize parameter to update the VFA, $\alpha$. It is going to be ignored when `USE_VFA` is set to `False`	
- `SAVE_PLOTS`:	Set to `True` if you want to save the plots to PDF files
- `URGENT_DEMAND_BONUS`: is the weight $c^{urgent}$, used in the utility function
- `ELECTIVE_DEMAND_BONUS`: is the weight $c^{elective}$, used in the utility function
- `DISCARD_BLOOD_PENALTY`: is the weight $c^{discard}$, used in the utility function
- `SHOW_PLOTS`: Set to `True` if you want to show plots of the figures described below, if `False` will just return dictionary of outputs.

In [39]:
param_dict = {
        'BLOOD_FOR_ELECTIVE_PENALTY': 0,
        'SURGE_PROB': 0.5,
        'NUM_TRAINNING_ITER': 1,
        'NUM_TESTING_ITER': 2,
        'USE_VFA': True,
        'ALPHA': 0.25,
        'SAVE_PLOTS': False,
        'URGENT_DEMAND_BONUS': 10, 
        'ELECTIVE_DEMAND_BONUS': 2.5,
        'DISCARD_BLOOD_PENALTY': -3,
        'SHOW_PLOTS': False
       }
# this line runs everything for the given parameters.
display(Main(param_dict))

********************Started Main*****************



{'Policy: ': ('VFA-Based', 'SURGE_0.5'),
 'name': 'PolicyVFA-Based_SURGE_0.5_PEN_0_ALPHA_0.25',
 'coverage': 'Average Coverage: -  Urgent: 0.95 Elective: 0.77 Avg: 0.85',
 'Percentage of blood discarded: ': 0.6907545164718385,
 'Average Total Demand (urgent): ': 284.5,
 'Average Total Demand Uncovered (urgent): ': 32.5,
 'Average Total Demand (elective): ': 327.5,
 'Average Total Demand Uncovered (elective): ': 111.0,
 'Average units of blood discarded: ': 6.5,
 'Average total contribution during TESTING iterations': 6321.75,
 'StDev total contribution during TESTING iterations': 238.25,
 'Average utility: ': 6321.75,
 'StDev utility': 238.25}

In [34]:
np.std([1])

0.0